# Load Libraries

In [2]:
# pip3 install s3fs
from sagemaker import Session
import sagemaker
import boto3
import re
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import os

In [3]:
role = get_execution_role()

bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'customer/notebook/output'
output_path =  f's3://{bucket_name}/{prefix_name}'
data_location = f"s3://{bucket_name}/{prefix_name}/train"

train_data_file = f's3://{bucket_name}/{prefix_name}/train/train_data.csv'

print("---------------------------------")
print(f"bucket_name '{bucket_name}':")
print(f"prefix_name '{prefix_name}':")
print(f"data_location '{data_location}':")
print(f"output_path '{output_path}':")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
---------------------------------
bucket_name 'eliezerraj-908671954593-dataset':
prefix_name 'customer/notebook/output':
data_location 's3://eliezerraj-908671954593-dataset/customer/notebook/output/train':
output_path 's3://eliezerraj-908671954593-dataset/customer/notebook/output':


In [4]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()

#df_data = pd.read_csv(train_data_file, skiprows = 1, header=None)
#df_data_scaled = scaler.fit_transform(df_data).astype('float32')
#df_data_scaled

# Id the dataset is already scaled
df_data = pd.read_csv(train_data_file, header=None)
df_data_scaled = df_data.to_numpy().astype('float32')
df_data_scaled

array([[-0.16540559,  0.50336814, -0.5026349 ,  0.6200389 ],
       [ 0.3335704 ,  2.0431986 ,  0.8342485 , -0.736292  ],
       [ 0.58305836,  0.50336814,  0.8342485 ,  1.2982044 ],
       ...,
       [-0.29014957, -1.0364624 , -0.5026349 , -0.736292  ],
       [-2.0365655 , -0.26654714,  0.8342485 , -0.05812655],
       [-0.41489357, -0.26654714,  0.8342485 , -0.736292  ]],
      dtype=float32)

# Build Model

In [5]:
from sagemaker import KMeans

kmeans = KMeans(role=role,
                instance_count=1,
                instance_type='ml.m4.xlarge',
                data_location=data_location,
                output_path=output_path,
                init_method='kmeans++',
                k=5,
                epochs=5)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [6]:
kmeans.fit(kmeans.record_set(df_data_scaled))

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: kmeans-2024-04-27-02-42-52-576


2024-04-27 02:42:52 Starting - Starting the training job...
2024-04-27 02:43:06 Starting - Preparing the instances for training...
2024-04-27 02:43:42 Downloading - Downloading input data...
2024-04-27 02:44:12 Downloading - Downloading the training image.........
2024-04-27 02:45:33 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/27/2024 02:45:50 INFO 140070763009856] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'init_method': 'random', 'mini_batch_size': '5000', 'epochs': '1', 'extra_center_factor': 'auto', 'local_lloyd_max_iter': '300', 'local_lloyd_tol': '0.0001', 'local_lloyd_init_method': 'kmeans++', 'local_lloyd_num_trials': 'auto', 'half_life_time_size': '0', 'eval_metrics': '["msd"]', 'force_dense': 'true', '_disable_wait_to_read': 'false', '_enable_profiler': 'false', '_kvstore': 'auto'

# Deploy model - create endopoint

In [7]:
job_name = kmeans.latest_training_job.name

model_output_path =  f's3://{bucket_name}/{prefix_name}/{job_name}/output/model.tar.gz'

print("model_output_path : ",model_output_path)

model_output_path :  s3://eliezerraj-908671954593-dataset/customer/notebook/output/kmeans-2024-04-27-02-42-52-576/output/model.tar.gz


In [8]:
# Get the model
from time import gmtime, strftime

client = boto3.client(service_name="sagemaker")
region = boto3.Session().region_name

model_name = "kmeans-serverless-customer-v1-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)

Model name: kmeans-serverless-customer-v1-2024-04-27-02-46-45


In [9]:
# Get the container image
from sagemaker import image_uris
container = image_uris.retrieve("kmeans", region, "latest")

print("containere: " + container)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


containere: 404615174143.dkr.ecr.us-east-2.amazonaws.com/kmeans:1


In [10]:
# dummy environment variables
byo_container_env_vars = {"SAGEMAKER_CONTAINER_LOG_LEVEL": "20", "SOME_ENV_VAR": "myEnvVar"}

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": container,
            "Mode": "SingleModel",
            "ModelDataUrl": model_output_path,
            "Environment": byo_container_env_vars,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response["ModelArn"])

Model Arn: arn:aws:sagemaker:us-east-2:908671954593:model/kmeans-serverless-customer-v1-2024-04-27-02-46-45


In [12]:
# Serveless endpoint config
model_epc_name = "mlops-serverless-epc-customer-v1-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=model_epc_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            },
        },
    ],
)

# EC2 endpoint
kmeans_predictor = kmeans.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-2:908671954593:endpoint-config/mlops-serverless-epc-customer-v1-2024-04-27-02-49-56


In [13]:
# Serveless endpoint

endpoint_name = "kmeans-serverless-ep-customer-v1" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=model_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-2:908671954593:endpoint/kmeans-serverless-ep-customer-v12024-04-27-02-56-37


# Validation

In [16]:
result = endpoint_name.predict(dt_data_scaled)
clusters = [r.label["closest_cluster"].float32_tensor.values[0] for r in result]

AttributeError: 'str' object has no attribute 'predict'

In [137]:
df_data_raw = pd.read_csv(train_data_file)
#df_data_raw.head(3)

In [146]:
df_result_final = df_data_raw[['Customer_Age',
                               'Dependent_count',
                               'Education_Level_Quality', 
                               'Income_Category_Quality']]
df_result_final["CLUSTER"] = clusters
df_result_final.head(10)

,Customer_Age,Dependent_count,Education_Level_Quality,Income_Category_Quality,CLUSTER
0,45,3,1,3,1.0
1,49,5,3,1,3.0
2,51,3,3,4,2.0
3,40,4,1,1,1.0
4,40,3,0,3,1.0
5,44,2,3,2,3.0
6,51,4,0,5,2.0
7,32,0,1,3,4.0
8,37,3,0,3,1.0
9,48,2,3,4,2.0


# Add a new data

In [139]:
# new data customes
df_new_customer_data = [{ 'Customer_Age': 20, 
                         'Dependent_count': 1, 
                         'Education_Level_Quality': 1, 
                         'Income_Category_Quality': 1 }]

df_new_customer_data = pd.DataFrame(df_new_customer_data)
df_new_customer_data

,Customer_Age,Dependent_count,Education_Level_Quality,Income_Category_Quality
0,20,1,1,1


In [144]:
df_customer_test = pd.concat([df_new_customer_data, df_data_raw], axis=0)
df_customer_test.head(3)

,Customer_Age,Dependent_count,Education_Level_Quality,Income_Category_Quality
0,20,1,1,1
0,45,3,1,3
1,49,5,3,1


In [141]:
df_new_customer_data_scaled_02 = scaler.fit_transform(df_customer_test).astype('float32')
df_new_customer_data_scaled_02

array([[-3.2820961 , -1.0363563 , -0.50260377, -0.736236  ],
       [-0.16500166,  0.50346863, -0.50260377,  0.62012565],
       [ 0.33373347,  2.0432935 ,  0.8343289 , -0.736236  ],
       ...,
       [-0.28968543, -1.0363563 , -0.50260377, -0.736236  ],
       [-2.0352583 , -0.26644385,  0.8343289 , -0.05805517],
       [-0.41436923, -0.26644385,  0.8343289 , -0.736236  ]],
      dtype=float32)

In [142]:
new_customer_result_02 = kmeans_predictor.predict(df_new_customer_data_scaled_02)
new_customer_clusters_02 = [r.label["closest_cluster"].float32_tensor.values[0] for r in new_customer_result_02]

In [148]:
df_customer_test["CLUSTER"] = new_customer_clusters_02
df_customer_test.head(11)

,Customer_Age,Dependent_count,Education_Level_Quality,Income_Category_Quality,CLUSTER
0,20,1,1,1,4.0
0,45,3,1,3,1.0
1,49,5,3,1,3.0
2,51,3,3,4,2.0
3,40,4,1,1,1.0
4,40,3,0,3,1.0
5,44,2,3,2,3.0
6,51,4,0,5,2.0
7,32,0,1,3,4.0
8,37,3,0,3,1.0


In [149]:
print(kmeans_predictor.endpoint)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


kmeans-2024-04-26-02-18-34-583


In [150]:
kmeans_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: kmeans-2024-04-26-02-18-34-583
INFO:sagemaker:Deleting endpoint with name: kmeans-2024-04-26-02-18-34-583
